## Iterators, generators and coroutines

<a href='https://docs.python.org/3/library/itertools.html' target='_blank'>Python standard library: Itertools</a>


<a href="http://www.dabeaz.com/finalgenerator/" target='_blank'>David Beazley's PyCon'14 presentation</a>

!!! info ""
    
    `Simple Generators` <badge-pep nr='255'></badge-pep>
    : The proposal for adding generators and the yield statement to Python.
    
    `Coroutines via Enhanced Generators` <badge-pep nr='342'></badge-pep>
    : The proposal to enhance the API and syntax of generators, making them usable as simple coroutines.
    
    `Syntax for Delegating to a Subgenerator` <badge-pep nr='380'></badge-pep>
    : The proposal to introduce the <code>yield_from</code> syntax, making delegation to sub-generators easy.

In [14]:
def echo(value=None):
    print("Execution starts when 'next()' is called for the first time.")
    try:
        while True:
            try:
                value = (yield value)
            except Exception as e:
                value = e
    finally:
        print("Don't forget to clean up when 'close()' is called.")

generator = echo(1)
print(next(generator))

Execution starts when 'next()' is called for the first time.
1


In [15]:
print(next(generator))

None


In [16]:
print(generator.send(2))

2


In [17]:
generator.throw(TypeError, "spam")

TypeError('spam')

In [18]:
generator.close()

Don't forget to clean up when 'close()' is called.


In [19]:
def gen():  # defines a generator function
    yield 123

In [3]:
async def agen(): # defines an asynchronous generator function (PEP 525)
    yield 123j

In [62]:
%timeit list(gen())

709 ns ± 8.22 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
await agen()

TypeError: object async_generator can't be used in 'await' expression

In [23]:
import asyncio 

async def hello():
    await asyncio.sleep(3)
    print("hello")

In [25]:
await hello()
print("Finished")

Finished


In [9]:
from asyncio import get_running_loop

In [10]:
loop = get_running_loop()

In [12]:
loop.

RuntimeError: This event loop is already running

In [8]:
%autoawait


agen()

IPython autoawait is `on`, and set to use `asyncio`


TypeError: object async_generator can't be used in 'await' expression

## Simple Generators

In [65]:
def echo(value=None):
    print("Execution starts when 'next()' is called for the first time.")
    try:
        while True:
            try:
                value = (yield value)
            except Exception as e:
                value = e
    finally:
        print("Don't forget to clean up when 'close()' is called.")

In [66]:
generator = echo(1)

In [67]:
next(generator)

Execution starts when 'next()' is called for the first time.


1

In [68]:
generator.close()

Don't forget to clean up when 'close()' is called.


In [70]:
next(generator)

StopIteration: 

Generator have to be first initialized by <code>next</code> statement unless a <code>TypeError</code> is raised. The idea in initializing the generator is to get it reach first the yield statement. From that on, <code>next()</code> function and <code>.send()</code> method can be used.

In [73]:
generator = echo(1)
generator.send(2)

TypeError: can't send non-None value to a just-started generator

In [74]:
generator.throw(TypeError, "spam")

TypeError: spam

Simple decorator for initializing a generator

In [153]:
def initgen(func):
    def wrapper(*args,**kwargs):
        gen = func(*args,**kwargs)
        next(gen)
        return gen
    return wrapper

In [138]:
@initgen
def echo2(value=None):
    print("Execution starts when 'next()' is called for the first time.")
    try:
        while True:
            try:
                value = (yield value)
            except Exception as e:
                value = e
    finally:
        print("Don't forget to clean up when 'close()' is called.")

In [139]:
generator = echo2(1)
r = generator.send(2)
print(r)

Execution starts when 'next()' is called for the first time.
2


In [140]:
r = generator.send(4)
print(r)

4


In [141]:
print(_)

Moyenne 13.0


In [142]:
@initgen
def echo3(value=None):
    print("Execution starts when 'next()' is called for the first time.")
    try:
        while True:
            try:
                A = (yield value) + 10
            except Exception as e:
                value = e
    finally:
        print("Don't forget to clean up when 'close()' is called.")

In [143]:
generator = echo3(1)
r = generator.send(2)
print(r)

Execution starts when 'next()' is called for the first time.
Don't forget to clean up when 'close()' is called.
1


In [144]:
generator.send(4)

1

In [145]:
@initgen
def moyenne():
    _sum = 0
    _i = 0
    _moy = 0
    while True:
        elt = (yield f"Moyenne {_moy}")
        _sum += elt
        _i += 1
        _moy = _sum / _i

In [146]:
m = moyenne()

In [147]:
m.send(4)

'Moyenne 4.0'

In [148]:
m.send(20)

'Moyenne 12.0'

In [168]:
@initgen
def moyenne2():
    _sum = 0
    _i = 0
    _moy = 0
    while True:
        elt = (yield f"Moyenne {_moy}" + "...") + 10
        _sum += elt
        _i += 1
        _moy = _sum / _i

In [169]:
m2 = moyenne2()

In [170]:
m2.send(4)

'Moyenne 14.0...'

In [171]:
m2.send(2)

'Moyenne 13.0...'

In [172]:
m2.close()

In [183]:
def fibonacci(n):
    """ A generator for creating the Fibonacci numbers """
    a, b, counter = (0, 1, 0)
    while True:
        if (counter > n): 
            return
        yield a
        a, b = b, a + b
        counter += 1
f = fibonacci(5)
for x in f:
    print(x, "-> ", end="", flush=True)

0 -> 1 -> 1 -> 2 -> 3 -> 5 -> 